In [ ]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

In [ ]:
url = "https://soundcloud.com/discover"

In [ ]:
browser = webdriver.Chrome()
browser.get(url)

In [ ]:
html = browser.page_source
soup = BeautifulSoup(html)

In [ ]:
tracks = []
playlists = []

users = []
# Lưu link profile của producer
urls = []

In [ ]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_down_till_end():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

In [ ]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_up():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(document.body.scrollHeight, 0);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(document.body.scrollHeight,0);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

In [ ]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, sep='\t')

In [ ]:
# Đồng ý cookies
cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
cookies.click()
    
# Chọn một tab genre (demo)
time.sleep(2)
genre = browser.find_element(By.XPATH,"//a[@class='playableTile__heading playableTile__mainHeading sc-truncate sc-type-light sc-text-secondary sc-font-light sc-link-dark sc-link-primary sc-text-h4']")
genre.click()

time.sleep(2)
scroll_down_till_end()

In [ ]:
html = browser.page_source
soup = BeautifulSoup(html)

# Trích xuất link profile của các producer có mặt trong 1 tab genre
time.sleep(3)
search = soup.find_all("li", "systemPlaylistTrackList__item")
time.sleep(3)
for producer in search:
    url = producer.find("a", "trackItem__username").get('href')
    #print(name.get('href'))
    urls.append(url)

**Lấy tất cả tracks của một user**

In [ ]:
def get_all_tracks():
    scroll_down_till_end()
    time.sleep(2)
    html = browser.page_source
    soup = BeautifulSoup(html)
    
    songs = soup.find_all("li", "soundList__item")
    for track in songs:
        title = track.find("a", "soundTitle__title").get_text().strip()
        # kiểm tra có bị trùng
        if title not in tracks:
            tracks.append(title)

**Lấy tất cả playlists của một user**

In [ ]:
def get_all_playlists():
    time.sleep(3)
    tab_playlists = browser.find_element(By.XPATH,'//a[contains(@href,"/sets")]')
    tab_playlists.click()
    
    time.sleep(3)
    scroll_down_till_end()
    html = browser.page_source
    soup = BeautifulSoup(html)
    
    playlists = soup.find_all("li", "soundList__item")
    for playlist in playlists:
        title = playlist.find("a", "soundTitle__title").
        if title is None:
            break
        # kiểm tra có bị trùng
#         if title not in playlists:
        print(title)
        playlists.append(str(title.get_text()))

In [ ]:
def parse_tracks_per_genre():
    for url in urls:   
        yup = browser.find_element(By.XPATH,"//a[@href='{link}']".format(link = url))
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        get_all_tracks()
        time.sleep(2)
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

In [ ]:
parse_tracks_per_genre()

**Lưu data vào file tsv**

In [ ]:
outputFile = "playlists.tsv"
list_to_tsv_csv(playlists, outputFile)
print(len(playlists))